In [1]:
import json

In [2]:
import openai
from   pprint import pprint
import requests
from   tenacity import retry, wait_random_exponential, stop_after_attempt
from   termcolor import colored

In [3]:
from server.senzing_api import SenzingAPI

# Config

In [4]:
if openai.api_key:
    print('OpenAI API Key: Found')
else:
    print(colored('OpenAI API Key: Not Found', 'red'))

OpenAI API Key: Found


In [5]:
CHAT_MODEL  = 'gpt-4-0613'
TEMPERATURE = 0.7
MAX_TOKENS  = 1024

# Functions

In [6]:
FUNCTIONS = [{
    'name':        'entity_details',
    'description': 'Retrieve entity data based on the ID of a resolved identity.',
    'parameters':  {
        'type': 'object',
        'properties': {
            'entity_id': {'type': 'integer'},
        },
        'required': ['entity_id'],
    },
},{
    'name':        'entity_how',
    'description': 'Determines and details steps-by-step how records resolved to an ENTITY_ID.',
    'parameters':  {
        'type': 'object',
        'properties': {
            'entity_id': {'type': 'integer'},
        },
        'required': ['entity_id'],
    },
},{
    'name':        'entity_report',
    'description': 'Return 10 entities with either matches, possible matches, or relationships.',
    'parameters':  {
        'type': 'object',
        'properties': {
            'export_flags': {'type': 'string', 'enum': ["MATCHED", "POSSIBLE_MATCHES", "POSSIBLE_RELATIONSHIPS"]},
        },
        'required': ['export_flags'],
    },
},{
    'name':        'entity_search',
    'description': 'Retrieves entity data based on a user-specified set of entity attributes.',
    'parameters':  {
        'type': 'object',
        'properties': {
            'attrs': {
                'type': 'object',
                'properties': {
                    'ADDR_CITY': {'type': 'string'},
                    'ADDR_COUNTRY': {'type': 'string'},
                    'ADDR_FULL': {'type': 'string'},
                    'ADDR_LINE1': {'type': 'string'},
                    'ADDR_POSTAL_CODE': {'type': 'string'},
                    'ADDR_STATE': {'type': 'string'},
                    'DATE_OF_BIRTH': {'type': 'string'},
                    'DRIVERS_LICENSE_NUMBER': {'type': 'string'},
                    'EMAIL_ADDRESS': {'type': 'string'},
                    'NAME_FIRST': {'type': 'string'},
                    'NAME_FULL': {'type': 'string'},
                    'NAME_LAST': {'type': 'string'},
                    'NAME_MIDDLE': {'type': 'string'},
                    'NAME_ORG': {'type': 'string'},
                    'NAME_SUFFIX': {'type': 'string'},
                    'NATIONAL_ID_NUMBER': {'type': 'string'},
                    'PASSPORT_COUNTRY': {'type': 'string'},
                    'PASSPORT_NUMBER': {'type': 'string'},
                    'PHONE_NUMBER': {'type': 'string'},
                    'SSN_NUMBER': {'type': 'string'},
                },
            },
        },
        'required': [],
    },
}]

# GPT Utils

In [7]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages,
                            functions = FUNCTIONS,
                            function_call = None,
                            model = CHAT_MODEL,
                            temperature = TEMPERATURE,
                            max_tokens  = MAX_TOKENS,
                           ):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [8]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    formatted_messages = []
    for message in messages:
        if message["role"] == "system":
            formatted_messages.append(f"system: {message['content']}\n")
        elif message["role"] == "user":
            formatted_messages.append(f"user: {message['content']}\n")
        elif message["role"] == "assistant" and message.get("function_call"):
            formatted_messages.append(f"assistant: {message['function_call']}\n")
        elif message["role"] == "assistant" and not message.get("function_call"):
            formatted_messages.append(f"assistant: {message['content']}\n")
        elif message["role"] == "function":
            formatted_messages.append(f"function ({message['name']}): {message['content']}\n")
    for formatted_message in formatted_messages:
        print(
            colored(
                formatted_message,
                role_to_color[messages[formatted_messages.index(formatted_message)]["role"]],
            )
        )

# Senzing API

In [9]:
api = SenzingAPI()

In [10]:
def function_call(msg):
    assert 'assistant' == msg['role']
    assert not msg['function_call'] is None
    
    func_name = msg['function_call']['name']
    func_args = json.loads(msg['function_call']['arguments'])
    func_resp = getattr(api, func_name)(**func_args)
    
    func_msg = dict(
        role = 'function',
        name = func_name,
        content = json.dumps(func_resp),
    )
    
    return func_msg

# Test

In [11]:
def chat(messages):
    chat_response = chat_completion_request(messages)

    try:
        # Filter: Remove Function Call Messages
        messages = [msg for msg in messages
                    if  msg['role'] in {'system', 'user', 'assistant'}
                    and msg['content']
                   ]
        
        assistant_message = chat_response.json()["choices"][0]["message"]
        messages.append(assistant_message)

        pretty_print_conversation(messages)
    except Exception:
        pprint(chat_response.json())
    
    return messages

In [12]:
messages = [dict(role='user', content='Show me examples of matches')]
messages = chat(messages)

user: Show me examples of matches

assistant: {'name': 'entity_report', 'arguments': '{\n  "export_flags": "MATCHED"\n}'}



In [13]:
function_message = function_call(messages[-1])
messages.append(function_message)

messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.



In [14]:
messages.append(dict(role='user', content='Show me details on Marie Kusha'))
messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.

user: Show me details on Marie Kusha

assistant:

In [15]:
function_message = function_call(messages[-1])
messages.append(function_message)

messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.

user: Show me details on Marie Kusha

assistant:

In [16]:
messages.append(dict(role='user', content='How did this entity come together?'))
messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.

user: Show me details on Marie Kusha

assistant:

In [17]:
function_message = function_call(messages[-1])
messages.append(function_message)

messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.

user: Show me details on Marie Kusha

assistant:

In [18]:
messages.append(dict(role='user', content='Who else lives at her address?'))
messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.

user: Show me details on Marie Kusha

assistant:

In [19]:
function_message = function_call(messages[-1])
messages.append(function_message)

messages = chat(messages)

user: Show me examples of matches

assistant: Here are some examples of matched entities:

1. Entity: Marie Kusha (Entity ID: 1)
   - Data sources: Customers
   - Record IDs: 1016, 1017, 1018, 1015
   - Related entities: Eddie Knight (Entity ID: 32), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

2. Entity: Robert Smith (Entity ID: 2)
   - Data sources: Customers
   - Record IDs: 1004, 1003, 1001, 1002
   - Related entities: Robert E Smith Sr (Entity ID: 38), Robert Smith (Entity ID: 200001), Patricia Smith (Entity ID: 200025)

3. Entity: Muhammed Antoun (Entity ID: 3)
   - Data sources: Customers
   - Record IDs: 1022, 1023

4. Entity: Eddie Knight (Entity ID: 32)
   - Data sources: Customers, Watchlist
   - Record IDs: 1010, 1011, 1009, 1014, 1012
   - Related entities: Marie Kusha (Entity ID: 1), Mark Kusha (Entity ID: 41), Marsha Kusha (Entity ID: 42)

Please, let me know if you need more information about these entities.

user: Show me details on Marie Kusha

assistant: